# Document Clustering and Topic Modeling

In this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 0: Setup Google Drive Environment

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file = drive.CreateFile({'id':'1cQ81XJ0zduruIM2CkDTkHTuFP3ngKpad'}) 
file.GetContentFile('watch_reviews.tsv')  # tab separate

# Part 1: Load Data

In [ ]:
import numpy as np
import pandas as pd
import nltk
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load data into dataframe
df = pd.read_csv('watch_reviews.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [ ]:
# Remove missing value
df.dropna(subset=['review_body'],inplace=True)

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [ ]:
# use the first 400,002 data as our training data
data = df.loc[:400001, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [ ]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("n't")
stopwords.append("br")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:11])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've"]


Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [ ]:
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'watch',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'wear',
 'dainti']

In [ ]:
data[0]

'Absolutely love this watch! Get compliments almost every time I wear it. Dainty.'

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

***example:***
(1,2)
dictionary :[Arthur, da, Jason, huang, arthur da, da jason, jason da, da da, da huang]

document1: "Arthur da Jason"

document 2: "Jason da da huang"

document1: tf-idf [1, 0.5, 0.5, 0];  document2: tf-idf [0, 1, 0.5, 1]  

2-gram: 

document 1: Arthur da, da Jason; document 2: Jason da, da da, da huang bigram

3-gram:

document 1: Athur da Jason;  document 2: Jason da da, da da huang

[Arhur, da, Jason...]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 400002 reviews and 218 terms.


In [ ]:
# check the parameters
tfidf_model.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.99,
 'max_features': 1000,
 'min_df': 0.01,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenization_and_stemming>,
 'use_idf': True,
 'vocabulary': None}

In [ ]:
tfidf_matrix.todense()

matrix([[0.        , 0.50544265, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.10080303,
         0.        ]])

Save the terms identified by TF-IDF.

In [ ]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [ ]:
# print out words
tf_selected_words

['abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'appear',
 'arriv',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'bracelet',
 'brand',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'casual',
 'chang',
 'cheap',
 'christma',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'coupl',
 'cute',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'durabl',
 'easi',
 'easili',
 'eleg',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'far',
 'fast',
 'featur',
 'feel',
 'fine',
 'fit',
 'friend',
 'function',
 'gave',
 'gift',
 'glass',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi',
 'high',
 'hold',
 'hope',
 'hour',
 'howev',
 'h

# Part 4: K-means clustering

In [ ]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [ ]:
# create DataFrame films from all of the input files.
product = { 'review': df[:400002].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [ ]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,1
1,I love this watch it keeps time wonderfully.,0
2,Scratches,1
3,"It works well on me. However, I found cheaper ...",1
4,Beautiful watch face. The band looks nice all...,1
5,"i love this watch for my purpose, about the pe...",1
6,"for my wife and she loved it, looks great and ...",3
7,I was about to buy this thinking it was a Swis...,1
8,Watch is perfect. Rugged with the metal &#34;B...,3
9,Great quality and build.<br />The motors are r...,3


In [ ]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
1,266629
3,41453
4,34206
0,31075
2,26639


In [ ]:
km.cluster_centers_

# 239数的list -> cluster 0的中心点的tf-idf值
#-> assumption: 中心点的值可以代表这个cluster
#-> tf-idf值越大，对应的词越能代表这个document
#-> 选出了tf-idf最大的6个值对应的词来代表这个cluster

array([[0.00056609, 0.0166695 , 0.00057423, ..., 0.00178175, 0.00407686,
        0.00604503],
       [0.00456926, 0.00728066, 0.00679901, ..., 0.00934325, 0.02342758,
        0.01781092],
       [0.00044888, 0.00103826, 0.00254973, ..., 0.00421343, 0.00876541,
        0.00242157],
       [0.00098834, 0.00436875, 0.00373491, ..., 0.00400723, 0.0098775 ,
        0.0088626 ],
       [0.00040478, 0.00025395, 0.00191732, ..., 0.00253423, 0.00386537,
        0.00219547]])

In [ ]:
km.cluster_centers_.shape

(5, 218)

In [ ]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :5]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%config
--NotebookApp.iopub_data_rate_limit

Available objects for config:
     Shell
     ColabHistoryManager
     PrefilterManager
     IPCompleter
     DisplayFormatter
     MagicsManager
     ScriptMagics
     AliasManager
     IPKernelApp
     InlineBackend
     StoreMagics
     StoreMagics


NameError: ignored

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [ ]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [ ]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(400002, 5)
[[0.31053225 0.05803039 0.51442885 0.05929335 0.05771516]
 [0.07620186 0.07593558 0.48526387 0.28499111 0.07760758]
 [0.10000003 0.10078978 0.10000003 0.5992101  0.10000005]
 ...
 [0.04609353 0.04712218 0.04668799 0.81406335 0.04603295]
 [0.07549316 0.07627932 0.07456301 0.50124145 0.27242307]
 [0.03094715 0.03136021 0.03070797 0.87564428 0.03134038]]


In [ ]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 239)
[[ 0.45702595  1.20639616  2.70011872 ...  3.42921873 11.98291702
  10.59045232]
 [ 1.11878558  0.20171755  0.20206997 ...  0.20064438  0.20157287
   0.25692955]
 [ 0.20018766  6.58079413  0.20019734 ...  2.49904592  0.20111408
   0.20018412]
 [ 3.03009066  0.20066258  0.20478698 ...  0.20425652  3.24232094
   0.25208876]
 [ 0.20061381  0.20004266  0.20023367 ...  0.55156965  0.20357684
   1.04003836]]


In [ ]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.31,0.06,0.51,0.06,0.06,2
Doc1,0.08,0.08,0.49,0.28,0.08,2
Doc2,0.10,0.10,0.10,0.60,0.10,3
Doc3,0.06,0.06,0.06,0.74,0.07,3
Doc4,0.04,0.35,0.04,0.53,0.04,3
Doc5,0.71,0.07,0.07,0.07,0.07,0
Doc6,0.42,0.06,0.07,0.06,0.38,0
Doc7,0.40,0.06,0.06,0.42,0.06,3
Doc8,0.05,0.05,0.05,0.81,0.05,3
Doc9,0.40,0.06,0.06,0.43,0.06,3


In [ ]:
df_document_topic['topic'].value_counts().to_frame()

,topic
3,138372
1,81847
0,66729
2,57730
4,55324


In [ ]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[3.54024852e-01 5.08821623e+02 7.78291368e-01 ... 4.88366978e+02
  2.00758484e+02 2.57657562e+02]
 [5.92213998e+01 2.37905248e+00 4.29510396e+02 ... 1.58508633e+03
  4.44095248e+03 2.31894708e+01]
 [2.01168241e-01 1.91898333e+03 2.01398986e-01 ... 2.01613725e-01
  2.04495853e-01 8.61247422e+02]
 [1.24370851e+03 2.47309519e+02 1.47411338e+03 ... 8.38743443e+02
  2.50726406e+03 4.29718390e+03]
 [2.02352940e-01 2.00860431e-01 2.15409411e+02 ... 2.05423172e-01
  2.00889947e-01 5.57362614e+00]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,anoth,appear,arriv,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,bracelet,brand,broke,button,buy,ca,came,case,casio,casual,chang,cheap,christma,clasp,...,small,smaller,solid,someth,son,star,start,stop,strap,style,stylish,super,sure,tell,thank,thing,think,thought,time,timex,took,tri,turn,use,valu,want,watch,water,way,wear,week,weight,white,wife,wish,work,worn,worth,wrist,year
Topic0,0.354025,508.821623,0.778291,42.564667,0.354869,0.200415,28.238904,860.909373,2816.321235,1340.881861,971.232800,32.003770,485.184671,97.754218,0.213680,546.531974,0.205335,1312.346544,2744.387512,298.605721,2174.479585,77.203260,116.332761,328.923168,1340.940141,340.831127,0.201916,774.775875,958.044988,0.200485,3433.965668,257.715802,669.060035,92.680322,33.233535,3.009273,1.709597,12.897694,0.239147,0.438932,...,10.683598,0.543566,2.908864,89.792383,0.201225,70.639410,0.202839,0.201250,1214.998218,767.670069,0.202101,464.263604,89.744209,2.184313,5027.006491,215.528042,285.822564,137.596189,906.130350,520.569755,83.333167,14.175214,0.219063,210.479220,0.656925,1658.815811,5607.224571,0.204287,545.754837,961.722180,118.104508,0.235106,15.571317,3278.114871,62.433593,77.807801,0.939341,488.366978,200.758484,257.657562
Topic1,59.221400,2.379052,429.510396,251.048151,917.676518,0.200747,12.285132,18.200940,0.203569,1.500797,8.700571,562.918295,1.757811,0.350459,0.440123,3179.111541,0.202642,1291.580578,0.587093,798.388139,2595.711464,1419.227201,186.730196,219.853804,207.866310,3.875372,927.846485,71.521775,0.200422,0.211320,458.940238,354.575753,10.916087,329.555150,7.403998,1371.721862,0.842820,876.163440,0.201056,264.514218,...,3597.289082,1387.407647,1058.063667,164.300636,0.268094,90.422734,0.237071,0.200492,663.817190,729.731893,1946.570003,118.550468,187.066265,68.202910,0.200617,207.627503,478.699126,438.851874,1323.996685,44.424816,53.548190,3.347130,0.323960,722.942334,2200.538853,860.463605,8985.680373,222.060255,722.945523,2474.123747,15.908260,1501.696051,23.669258,0.204461,228.589277,434.034971,298.189205,1585.086329,4440.952476,23.189471
Topic2,0.201168,1918.983328,0.201399,16.741047,0.200770,0.200653,8.395068,0.201511,0.200765,0.201778,8.782725,0.265549,190.768671,4348.359525,0.200830,0.261494,0.201208,7961.770990,0.201004,1188.222423,1.295146,0.201100,0.201652,0.201792,2963.646357,1.321401,0.201107,0.206735,0.200736,0.200409,3.860370,0.749676,4.356841,0.201765,0.200967,0.201240,0.200746,0.201071,2222.663375,0.200795,...,0.201112,0.450121,0.200948,0.202586,3236.229321,1.330917,0.200830,0.200677,0.200930,55.496748,1.923828,0.201994,118.243629,0.248967,0.307877,0.202126,97.843208,0.986968,508.726065,0.200888,0.210915,0.200819,0.201104,0.499538,0.201175,2.490936,6809.228024,0.201090,2.520940,716.085143,0.201154,0.201099,0.201075,150.907876,0.201500,3.055003,0.203448,0.201614,0.204496,861.247422
Topic3,1243.708512,247.309519,1474.113382,1540.553785,1560.370006,1500.070926,1431.310249,926.665388,25.211665,1335.127507,1688.067926,876.725824,1290.254528,16.006493,1092.696818,8107.198720,4921.489135,1003.091240,476.728675,1677.182078,1450.388693,1634.800878,2152.357558,1085.771971,2694.008146,1587.447481,717.938040,714.171887,2214.578966,1742.023831,2451.308915,2079.648373,2056.209557,1845.597826,1636.584527,171.907046,2362.556417,1788.792655,3.850095,1351.770663,...,2010.814781,561.624786,439.284803,1407.507509,3.701727,2200.098326,1472.050801,2660.707623,2792.857657,835.736441,28.015696,493.143844,1479.232142,1915.300555,0.296341,2335.183379,1797.546855,1702.675746,9409.055806,1297.894814,1626.468701,1918.139265,1278.005717,5560.598567,119.106293,2092.913564,17161.875233,2090.848980,1681.683410,4261.412738,3274.878801,392.204560,1296.372896,8.693832,1502.186897,6645.462308,1244.221775,838.743443,2507.264064,4297.183904
Topic4,0.202353,0.200860,215.409411,0.205500,0.207076,0.200673,0.260732,0.200875,0.200556,0.204199,0.200882,0.201468,1532.314800,0.20

In [ ]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,watch,thank,love,husband,buy,wife,qualiti,fast,recommend,great,amaz,best,ship,big,high
Topic 1,good,watch,look,wrist,small,like,size,band,great,fit,nice,comfort,big,pretti,eleg
Topic 2,love,excel,beauti,gift,watch,awesom,like,son,bought,expect,compliment,daughter,christma,absolut,item
Topic 3,watch,time,band,work,look,like,use,batteri,day,year,wear,set,return,hand,replac
Topic 4,great,nice,watch,product,price,work,perfect,look,ok,good,qualiti,cheap,arriv,time,bad
